# Modeling: Подготовка датасетов для моделей и создание самих моделей

## 1. Импорт библиотек

Импортируем стандартные библиотеки для анализа данных, а также пользовательские функции для визуализации и первичного осмотра данных.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

try:
    from utils.paths import join_path as pj
except:
    from scooter_sharing_analysis.utils.paths import join_path as pj

## 2. Загрузка очищенных данных

Загружаем файл rides_weather_data.csv нормализованный после очистки.

In [2]:
# Настраиваем словари с параметрами чтения CSV: путь, кодировка, дата-колонки
rides_options = {
    "filepath_or_buffer": pj("data", "rides_weather_data.csv"),
    "encoding": "utf-8",
    "parse_dates": ["start_date", "end_date", "day_timestamp", "hour_timestamp"],
}

rides_data = pd.read_csv(**rides_options)

## 3. Создание факторов для моделей: временные, лаговые, погодные

## 3.1. Создание временных и лаговых факторов

Группируем данные по часовому ряду, получая спрос за каждый час (demand), добавляет новую колонку: 'hour_of_day'. Делим данные на тестовые и тренировочные с соотношение 1:4, добавляем лаговые колонки: 'lag_1h', 'lag_24h', 'mean_last_24h', 'mean_last_7d'

In [3]:
# Группировка данных и получение спроса
demand = rides_data.groupby("hour_timestamp").agg({"id": "count"}).rename(columns={"id": "demand"}).reset_index()

# Добавление новых колонки на основе "hour_timestamp"
demand["hour_of_day"] = demand["hour_timestamp"].dt.hour

# Создание лаговых признаков
demand["lag_1h"] = demand["demand"].shift(1)
demand["lag_24h"] = demand["demand"].shift(24)
demand["mean_last_24h"] = demand["lag_1h"].rolling(24).mean()
demand["mean_last_7d"] = demand["lag_1h"].rolling(24 * 7).mean()

# Удаление строк с Nan значениями в лагах
demand = demand.dropna()

# Разбиение данных на тестувую и тренировочную части
X_demand_train, X_demand_test, Y_demand_train, Y_demand_test = train_test_split(demand, demand["demand"], test_size=0.2, shuffle=False)

# Вывод первой 1000 строк
X_demand_train.head(1000)


,hour_timestamp,demand,hour_of_day,lag_1h,lag_24h,mean_last_24h,mean_last_7d
168,2023-04-29 09:00:00,31,9,24.0,28.0,34.375000,31.898810
169,2023-04-29 10:00:00,28,10,31.0,24.0,34.500000,31.970238
170,2023-04-29 11:00:00,31,11,28.0,18.0,34.666667,31.892857
171,2023-04-29 12:00:00,44,12,31.0,21.0,35.208333,31.916667
172,2023-04-29 13:00:00,47,13,44.0,25.0,36.166667,31.928571
...,...,...,...,...,...,...,...
1163,2023-06-09 22:00:00,65,22,61.0,28.0,40.666667,41.190476
1164,2023-06-09 23:00:00,37,23,65.0,27.0,42.208333,41.136905
1165,2023-06-10 00:00:00,17,0,37.0,6.0,42.625000,41.184524
1166,2023-06-10 01:00:00,12,1,17.0,2.0,43.083333,41.119048
